In [0]:
!unzip combo.zip
!mv combo/* .

In [0]:
import pandas as pd
import io
import matplotlib.pyplot as plt

#Read from files & create pandas dataframes
pop = pd.read_csv("ACS_17_5YR_DP05_with_ann.csv", encoding = 'ISO-8859-1')
econ = pd.read_csv("ACS_17_5YR_S2301_with_ann.csv", encoding = 'ISO-8859-1')
rent = pd.read_csv("ACS_17_5YR_B25061_with_ann.csv", encoding = 'ISO-8859-1')
transport = pd.read_csv("ACS_17_5YR_S0802_with_ann.csv", encoding = 'ISO-8859-1')
affiliate = pd.read_csv("Affiliate-City-to-Id2.csv", encoding = 'ISO-8859-1')
jobs = pd.read_csv("jobs.csv", encoding = 'ISO-8859-1')
houses = pd.read_csv("house_counts.csv", encoding = 'ISO-8859-1')
food = pd.read_csv("cities_food.csv", encoding = 'ISO-8859-1')
food.rename(columns={'p_HC01_VC03':'Pop'}, inplace=True)

#Append prefixes to column headers of different datasets
pop = pop.add_prefix('p_')
econ = econ.add_prefix('e_')
rent = rent.add_prefix('r_')
transport = transport.add_prefix('t_')
jobs = jobs[['p_GEO.id2', 'job_count']]
houses = houses[['p_GEO.id2', 'count']]
houses.rename(columns={'count':'house_count'}, inplace=True)
food = food[['p_GEO.id2','SNAPSPTH16','FMRKTPTH16',\
  'QUA_FMRKT_SNAP16','QUA_FMRKT_WIC16','QUA_FMRKT_WICCASH16','QUA_FMRKT_SFMNP16']]           

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,4,5,7,8,11,12,19,20,23,24,27,28,31,32,35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,67,68,75,76,79,80,83,84,87,88,91,92,95,96,99,100,101,103,104,107,108,115,116,117,119,120,123,124,131,132,133,135,136,139,140,143,144,147,148,151,152,155,156,159,160,163,164,167,168,171,172,175,176,179,180,183,184,187,188,191,192,195,196,199,200,203,204,207,208,211,212,215,216,219,220,223,224,227,228,231,232,235,236,239,240,243,244,247,248,251,252,253,255,256,259,260,263,264,267,268,271,272,275,276,279,280,281,283,284,287,288,291,292,295,296,299,300,303,304,307,308,311,312,315,316,319,320,323,324,327,328,331,332,335,336,339,340,343,344,347,348,349,351,352,355,356) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns 

In [0]:
#Merge population, economic, rent, and transportation data into one dataframe
#We left-merge, so we only use keys that appear in "merged" originally
merged = pd.concat([pop, econ, rent, transport], axis = 1)

#Need to drop first row which is the description of the column
#Also need to convert GeoID to an integer
merged = merged.iloc[1:]
merged['p_GEO.id2'] = merged['p_GEO.id2'].astype(int)

#Merge job data and food data onto dataframe
merged = merged.merge(jobs, how = 'left', on = 'p_GEO.id2')
merged = merged.merge(food, how = 'left', on = 'p_GEO.id2')

#Merging housing data causes duplicates because some cities have multiple house values, need to drop duplicates
houses = houses.drop_duplicates()
merged = merged.merge(houses, how = 'left', on = 'p_GEO.id2')

In [0]:
#Gets list of geoid's of affiliate cities
geoid = list(affiliate['Geoid'].astype(int))

# Minimum affiliate population is 6001, max pop is 8560072
minPop = 6001
maxPop = 8560072

#Filtering by population above 6000 because the smallest affiliate city has a population above 6000
merged = merged[merged['p_HC01_VC03'].astype(int) > minPop]
geoidCol = merged['p_GEO.id2'].astype(int)

#Splits data into two dataframes: affiliate cities and non affiliate cities
affPopFiltered = merged[geoidCol.isin(geoid)]
nonaffPopFiltered = merged[~geoidCol.isin(geoid)]

In [0]:
# Simpson's Diversity Index
''' (sum[n] (n * (n-1))) / (N * (N-1)) '''

# General races we consider
# p_HC01_VC54 : Estimate; RACE - One race - White
# p_HC01_VC55 : Estimate; RACE - One race - Black or African American
# p_HC01_VC56 : Estimate; RACE - One race - American Indian and Alaska Native
# p_HC01_VC61 : Estimate; RACE - One race - Asian
# p_HC01_VC69 : Estimate; RACE - One race - Native Hawaiian and Other Pacific Islander
# p_HC01_VC74 : Estimate; RACE - One race - Some other race

#Suffix of columns that contain race information
indices = [54, 55, 56, 61, 69, 74]
N = 0
sum_n = 0

#Looks through columns with race information and performs computation to find diversity index
for i in indices:
  N += pd.to_numeric(merged['p_HC01_VC'+str(i)], errors = 'coerce')
  sum_n += pd.to_numeric(merged['p_HC01_VC'+str(i)], errors = 'coerce')\
    * (pd.to_numeric(merged['p_HC01_VC'+str(i)], errors = 'coerce') - 1)
merged['Diversity'] = 1-(sum_n/(N*(N - 1)))

In [0]:
# Public transportation ratio
# t_HC02_EST_VC01 : Car, truck, or van -- drove alone; Estimate; Workers 16 years and over
# t_HC03_EST_VC01 : Car, truck, or van -- carpooled; Estimate; Workers 16 years and over
# t_HC04_EST_VC01 : Public transportation (excluding taxicab); Estimate; Workers 16 years and over

#Calculates ratio of people that take public transportation over the total number of people
#  who take public transportation, drive, and carpool
drive = pd.to_numeric(merged['t_HC02_EST_VC01'], errors = 'coerce')
carpool = pd.to_numeric(merged['t_HC03_EST_VC01'], errors = 'coerce')
public = pd.to_numeric(merged['t_HC04_EST_VC01'], errors = 'coerce')
merged['Public transportation proportion'] = \
  pd.Series((public/(public+drive+carpool)))

In [0]:
# Thresholding for affordable housing rent
def two_digit(x):
  x = str(x)
  if len(x) == 1:
    x = "0" + x
  return x

def get_state(row):
  label = row['p_GEO.display-label']
  if (';' in label):
    tokens = label.split('; ')
  else: 
    tokens = label.split(', ')
  return tokens[1]
state = merged.apply(get_state, axis=1)
merged['state'] = state

converters = {'state':str, 'min_wage':float}
min_wage_pd = pd.read_csv('minimum_wage.csv', converters=converters)
min_wages_map = pd.Series(min_wage_pd.min_wage.values, \
                          index=min_wage_pd.state).to_dict()

def get_min_wage(row):
  if row['state'] == 'District of Columbia':
    row['state'] = 'D.C.'
  return min_wages_map[row['state']]

# Housing Rent Brackets
# r_HD01_VD02 : Estimate; Total: - Less than $100
# r_HD01_VD03 : Estimate; Total: - $100 to $149
# r_HD01_VD04 : Estimate; Total: - $150 to $199
# r_HD01_VD05 : Estimate; Total: - $200 to $249
# r_HD01_VD06 : Estimate; Total: - $250 to $299
# r_HD01_VD07 : Estimate; Total: - $300 to $349
# r_HD01_VD08 : Estimate; Total: - $350 to $399
# r_HD01_VD09 : Estimate; Total: - $400 to $449
# r_HD01_VD10 : Estimate; Total: - $450 to $499
# r_HD01_VD11 : Estimate; Total: - $500 to $549
# r_HD01_VD12 : Estimate; Total: - $550 to $599
# r_HD01_VD13 : Estimate; Total: - $600 to $649
# r_HD01_VD14 : Estimate; Total: - $650 to $699
# r_HD01_VD15 : Estimate; Total: - $700 to $749
# r_HD01_VD16 : Estimate; Total: - $750 to $799
# r_HD01_VD17 : Estimate; Total: - $800 to $899
# r_HD01_VD18 : Estimate; Total: - $900 to $999
# r_HD01_VD19 : Estimate; Total: - $1,000 to $1,249
# r_HD01_VD20 : Estimate; Total: - $1,250 to $1,499
# r_HD01_VD21 : Estimate; Total: - $1,500 to $1,999
# r_HD01_VD22 : Estimate; Total: - $2,000 to $2,499
# r_HD01_VD23 : Estimate; Total: - $2,500 to $2,999
# r_HD01_VD24 : Estimate; Total: - $3,000 to $3,499
# r_HD01_VD25 : Estimate; Total: - $3,500 or more
per_inc_housing = 0.5 # up-in-the-air, percentage of monthly income for housing
mw_upper_threshold = 1.75 # $12 wage (Leslie) vs. $7.25 min wage (in Pittsburgh)
tax_consideration = 1 # amount left after tax
fact = per_inc_housing * mw_upper_threshold * tax_consideration
merged['min_wage'] = merged.apply(get_min_wage, axis=1)
# 8 hours a day, 20 workdays as a standard lower limit
merged['mw_monthly_amount'] = 8*20*merged['min_wage']
# computed upper threshold
merged['mw_monthly_threshold'] = fact * merged['mw_monthly_amount']

def get_affordable(row): # a lower bound
  threshold = row['mw_monthly_threshold']
  # rent includes brackets where the maximum is <= threshold
  if threshold < 100: 
    return 0
  elif threshold >= 100 and threshold < 150:
    max_x = 2
  elif threshold >= 150 and threshold < 200:
    max_x = 3
  elif threshold >= 200 and threshold < 250:
    max_x = 4
  elif threshold >= 250 and threshold < 300:
    max_x = 5
  elif threshold >= 300 and threshold < 350:
    max_x = 6
  elif threshold >= 350 and threshold < 400:
    max_x = 7    
  elif threshold >= 400 and threshold < 450:
    max_x = 8
  elif threshold >= 450 and threshold < 500:
    max_x = 9
  elif threshold >= 500 and threshold < 550:
    max_x = 10
  elif threshold >= 550 and threshold < 600:
    max_x = 11
  elif threshold >= 600 and threshold < 650:
    max_x = 12
  elif threshold >= 650 and threshold < 700:
    max_x = 13
  elif threshold >= 700 and threshold < 750:
    max_x = 14
  elif threshold >= 750 and threshold < 800:
    max_x = 15
  elif threshold >= 800 and threshold < 900:
    max_x = 16   
  elif threshold >= 900 and threshold < 1000:
    max_x = 17
  elif threshold >= 1000 and threshold < 1250:
    max_x = 18
  elif threshold >= 1250 and threshold < 1500:
    max_x = 19
  elif threshold >= 1500 and threshold < 2000:
    max_x = 20
  elif threshold >= 2000 and threshold < 2500:
    max_x = 21
  elif threshold >= 2500 and threshold < 3000:
    max_x = 22
  elif threshold >= 3000 and threshold < 3500:
    max_x = 23   
  else: # >= 3500
    max_x = 24
  house_bracket_columns = ["r_HD01_VD" + two_digit(x) for x in range(2,max_x+1)]
  house_sum = row[house_bracket_columns].astype('uint64').sum()
  return house_sum

merged['affordable'] = merged.apply(get_affordable, axis=1)

In [0]:
# Thresholding for travel time to work

# Time Travel Brackets for Public Transport
# t_HC04_EST_VC109 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - Less than 10 minutes
# t_HC04_EST_VC110 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 10 to 14 minutes
# t_HC04_EST_VC111 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 15 to 19 minutes
# t_HC04_EST_VC112 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 20 to 24 minutes
# t_HC04_EST_VC113 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 25 to 29 minutes
# t_HC04_EST_VC114 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 30 to 34 minutes
# t_HC04_EST_VC115 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 35 to 44 minutes
# t_HC04_EST_VC116 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 45 to 59 minutes
# t_HC04_EST_VC117 : Public transportation (excluding taxicab); Estimate; TRAVEL TIME TO WORK - 60 or more minutes

travel_time_threshold = 45 # in minutes
travel_columns = ['t_HC04_EST_VC1' + two_digit(x) for x in range(9, 18)]

def get_travel(row):
  vals = row[travel_columns].values
  if '-' in vals or 'N' in vals or '(X)' in vals:
    return ("","","")
  total_num = row[travel_columns].astype('float').sum()
  if travel_time_threshold < 10:
    return (0.0, total_num, 0.0)
  elif travel_time_threshold >= 10 and travel_time_threshold < 15:
    max_x = 9
  elif travel_time_threshold >= 15 and travel_time_threshold < 20:
    max_x = 10
  elif travel_time_threshold >= 20 and travel_time_threshold < 25:
    max_x = 11
  elif travel_time_threshold >= 25 and travel_time_threshold < 30:
    max_x = 12
  elif travel_time_threshold >= 30 and travel_time_threshold < 35:
    max_x = 13
  elif travel_time_threshold >= 35 and travel_time_threshold < 45:
    max_x = 14
  elif travel_time_threshold >= 45 and travel_time_threshold < 60:
    max_x = 15
  else: # >= 60
    max_x = 16
  good_num = row[['t_HC04_EST_VC1' + two_digit(x) for x in range(9, max_x+1)]] \
          .astype('float').sum()
  return (good_num, total_num, float(good_num) / float(total_num))

res = merged.apply(get_travel, axis=1)
good_time_number = [r[0] for r in res]
total_number = [r[1] for r in res]
travel_prop = [r[2] for r in res]

merged['good_num_time_transit'] = good_time_number
merged['total_num_time_transit'] = total_number
merged['good_prop_time_transit'] = travel_prop

In [0]:
# Merged Data
merged_city_data = merged[['p_GEO.id2',
                       'p_GEO.display-label',
                       'job_count', 
                       'house_count', 
                       'p_HC01_VC03',      # Estimate; SEX AND AGE - Total population
                       'e_HC04_EST_VC01',  # Unemployment rate; Estimate; Population 16 years and over
                       'e_HC03_EST_VC01',  # Employment/Population Ratio; Estimate; Population 16 years and over
                       'e_HC01_EST_VC36',  # Total; Estimate; POVERTY STATUS IN THE PAST 12 MONTHS - Below poverty level
                       'e_HC01_EST_VC47',  # Total; Estimate; EDUCATIONAL ATTAINMENT - Population 25 to 64 years - Bachelor's degree or higher
                       'Public transportation proportion', 
                       'Diversity', 
                       'affordable',
                       'good_prop_time_transit',
                       'SNAPSPTH16',       # SNAP-authorized stores/1,000 pop, 2016
                       'FMRKTPTH16',       # Farmers' markets/1,000 pop, 2016
                       'QUA_FMRKT_SNAP16', 
                       'QUA_FMRKT_WIC16', 
                       'QUA_FMRKT_WICCASH16', 
                       'QUA_FMRKT_SFMNP16']]
merged_city_data.columns = ['GeoID', 
                   'Place Name',
                   'Indeed job count', 
                   'Craigslist affordable house count', 
                   'Population',
                   'Unemployment rate', 
                   'Employment to population ratio',
                   'Number of people below the poverty level',
                   'Number of people with a bachelors degree or higher',
                   'Public transportation proportion',
                   'Simpsons diversity score',
                   'Affordable housing in market',
                   'Proportion of public transit to work under 30 minutes',
                   'SNAP-authorized Stores per 1000 people (2016)', 
                   'Farmers Markets per 1000 people (2016)',
                   'Farmer Markets with SNAP per 1000 people (2016)', 
                   'Farmer Markets with WIC per 1000 people (2016)', 
                   'Farmer Markets with WIC Cash per 1000 people (2016)', 
                   'Farmer Markets with SFMNP per 1000 people (2016)']

print(merged_city_data)

In [0]:
merged_city_data.to_csv("merged_city_data.tsv",sep="\t")

In [0]:
numeric_columns = list(merged_city_data.columns.values)[2:]
print(numeric_columns)

In [0]:
for col in numeric_columns:
  merged_city_data[col] = pd.to_numeric(merged_city_data[col], errors='coerce')

# Normalization by percentage or population
merged_city_data['Unemployment rate percentage'] = \
  merged_city_data['Unemployment rate'].astype(float) / 100

merged_city_data['Employment to population ratio percentage'] = \
  merged_city_data['Employment to population ratio'].astype(float) / 100

merged_city_data['Indeed job count normalized'] = \
  merged_city_data['Indeed job count'].astype(float) \
  / merged_city_data['Population'].astype(float)

merged_city_data['Craigslist affordable house count normalized'] = \
  merged_city_data['Craigslist affordable house count'].astype(float) \
  / merged_city_data['Population'].astype(float)

merged_city_data['Affordable housing in market normalized'] = \
        merged_city_data['Affordable housing in market'].astype(float) \
  / merged_city_data['Population'].astype(float)

merged_city_data['Number of people below the poverty level normalized'] = \
  merged_city_data['Number of people below the poverty level'].astype(float) \
  / merged_city_data['Population'].astype(float)

merged_city_data['Number of people with a bachelors degree or higher normalized'] = \
  merged_city_data['Number of people with a bachelors degree or higher'].astype(float) \
  / merged_city_data['Population'].astype(float)

In [0]:
rem = ['Unemployment rate',
       'Employment to population ratio',
       'Indeed job count',
       'Craigslist affordable house count',
       'Affordable housing in market',
       'Number of people below the poverty level',
       'Number of people with a bachelors degree or higher'
]
for r in rem:
  numeric_columns.remove(r)
numeric_columns += [
    'Unemployment rate percentage',
    'Employment to population ratio percentage',
    'Indeed job count normalized',
    'Craigslist affordable house count normalized',
    'Affordable housing in market normalized',
    'Number of people below the poverty level normalized',
    'Number of people with a bachelors degree or higher normalized']

# Min/max normalization for each numeric column
for col in numeric_columns:
  col_min = merged_city_data[col].min()
  col_max = merged_city_data[col].max()
  diff = col_max - col_min
  merged_city_data[col+" min_max_normalized"] = (merged_city_data[col] - col_min) / diff
 

In [0]:
# z-score normalization for each numeric column
for col in numeric_columns:
    merged_city_data[col+" z-score"] = (merged_city_data[col] - merged_city_data[col].mean())\
    / merged_city_data[col].std()

In [0]:
merged_city_data.to_csv("merged_city_data_normalized.csv") 